In [1]:
DATA_PATH = "data/exports/"

timeframe_array = ['1m','15m','1h','4h', '1d', '1w']
timeframe_mapping = {'1m': '1min', '15m': '15min', '1h': '1h', '4h': '4h', '1d': '1D', '1w': '7D'}

timeframe_array = [
                '1m','15m','1h','4h', '1d', '1w',
                #'1M'
                ]
symbol_array = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT','ICPUSDT', 'AVAXUSDT']

interval_to_ms = {
    "1m": 60_000,
    "3m": 3 * 60_000,
    "5m": 5 * 60_000,
    "15m": 15 * 60_000,
    "30m": 30 * 60_000,
    "1h": 60 * 60_000,
    "2h": 2 * 60 * 60_000,
    "4h": 4 * 60 * 60_000,
    "6h": 6 * 60 * 60_000,
    "8h": 8 * 60 * 60_000,
    "12h": 12 * 60 * 60_000,
    "1d": 24 * 60 * 60_000,
    "3d": 3 * 24 * 60 * 60_000,
    "1w": 7 * 24 * 60 * 60_000,
    "1M": 30 * 24 * 60 * 60_000  # approximate month (30 days)
}

# Init

In [2]:
import pandas as pd
import time
import os
import data.datasource.binance_api as ba 
import business.utils.trading_signals as ts
import business.utils.trading_indicators as ti
import presentation.plotter as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import threading
import concurrent.futures
import logging
import time


base_start_time_1m = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
base_start_time_other = int(pd.Timestamp("2015-01-01").timestamp() * 1000)


stop_flag = threading.Event()
pd.set_option('display.max_columns', None)


symbol_array = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'ICPUSDT', 'AVAXUSDT']
timeframe_array = ['1m', '15m', '1h', '4h', '1d', '1w']
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


In [3]:
def get_existing_data_path(symbol, interval):
    """Return the exact file path if the CSV exists (case-sensitive for intervals)."""
    filename = f"data/exports/{symbol}_{interval}_data.csv"
    return filename if os.path.exists(filename) else None

In [4]:
def get_last_timestamp_from_csv(filepath):
    """Retrieve the last timestamp from an existing CSV file."""
    df = pd.read_csv(filepath, usecols=["timestamp"])
    if df.empty:
        return None
    last_timestamp = df["timestamp"].iloc[-1]
    return int(pd.Timestamp(last_timestamp).timestamp() * 1000)

In [5]:

def export_historical_data(symbol, interval, start_time):
    if interval not in interval_to_ms:
        raise ValueError(f"Interval {interval} not supported.")

    delta = interval_to_ms[interval]

    all_data = []
    start_overall = time.time()
    iteration_count = 0

    while True:
        iteration_count += 1
        klines = ba.get_binance_klines(symbol, interval, start_time, limit=1000)
        if not klines:
            print("No more data returned from Binance.")
            break

        all_data += klines
        # Advance start_time by the appropriate millisecond delta for the interval.
        start_time = klines[-1][0] + delta

        elapsed = time.time() - start_overall
        latest_ts = klines[-1][0]
        latest_dt = pd.to_datetime(latest_ts, unit='ms')
        print(f"Iteration {iteration_count}: Latest timestamp: {latest_ts} ({latest_dt}), "
              f"Total records: {len(all_data)}, Elapsed time: {elapsed:.2f} sec")

        # If fewer than 'limit' records are returned, assume we've reached the end.
        if len(klines) < 1000:
            print(f"Iteration {iteration_count}: Last batch retrieved with {len(klines)} records. Ending extraction.")
            break

        # Respect Binance rate limits.
        time.sleep(0.5)

    # Convert collected data into a DataFrame.
    df = pd.DataFrame(all_data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
    df.set_index("timestamp", inplace=True)

    print("Data extraction complete. Sample data:")
    print(df.head())

    # Export to CSV.
    df.to_csv(f'data/exports/{symbol}_{interval}_data.csv', index=True)

In [6]:
def append_new_data(symbol, interval, start_time, csv_path):
    """Append new data to an existing CSV file and save."""
    # Fetch new data from Binance
    new_data = []
    iteration_count = 0
    start_overall = time.time()

    while True:
        iteration_count += 1
        klines = ba.get_binance_klines(symbol, interval, start_time, limit=1000)
        if not klines:
            print(f"{symbol} {interval}: No more data from Binance.")
            break

        new_data += klines
        start_time = klines[-1][0] + interval_to_ms[interval]

        elapsed = time.time() - start_overall
        latest_ts = klines[-1][0]
        latest_dt = pd.to_datetime(latest_ts, unit='ms')
        print(f"{symbol} {interval} | Iteration {iteration_count}: Latest timestamp: {latest_dt}, "
              f"Total new records: {len(new_data)}, Elapsed: {elapsed:.2f}s")

        if len(klines) < 1000:
            print(f"{symbol} {interval}: Last batch retrieved with {len(klines)} records. Reached end.")
            break

        time.sleep(0.5)  # Respect Binance rate limits

    if not new_data:
        print(f"{symbol} {interval}: No new data to append.")
        return

    # Convert collected data into a DataFrame
    df_new = pd.DataFrame(new_data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ])
    df_new["timestamp"] = pd.to_datetime(df_new["timestamp"], unit='ms')

    # Load existing data
    df_existing = pd.read_csv(csv_path)
    df_existing["timestamp"] = pd.to_datetime(df_existing["timestamp"])

    # Combine and remove duplicates
    df_combined = pd.concat([df_existing, df_new]).drop_duplicates(subset=["timestamp"]).sort_values(by="timestamp")

    # Save updated CSV
    df_combined.to_csv(csv_path, index=False)
    print(f"{symbol} {interval}: Data appended and saved. Total records: {len(df_combined)}")


In [7]:
def export_or_update(symbol, interval):
    """Check if export exists, update if yes, otherwise download from scratch."""
    csv_path = get_existing_data_path(symbol, interval)

    if csv_path:
        print(f"✅ {symbol} {interval}: Existing CSV found. Checking for updates...")
        last_timestamp = get_last_timestamp_from_csv(csv_path)
        if last_timestamp:
            print(f"📅 Last recorded timestamp: {pd.to_datetime(last_timestamp, unit='ms')}")
            append_new_data(symbol, interval, last_timestamp, csv_path)
        else:
            start_time = base_start_time_1m if interval == "1m" else base_start_time_other
            print(f"⚠️ {symbol} {interval}: CSV is empty, starting from {pd.to_datetime(start_time, unit='ms')}...")
            export_historical_data(symbol, interval, start_time)
    else:
        start_time = base_start_time_1m if interval == "1m" else base_start_time_other
        print(f"🚀 {symbol} {interval}: No existing data. Starting from {pd.to_datetime(start_time, unit='ms')}...")
        export_historical_data(symbol, interval, start_time)

In [8]:
def merge_timeframes():
    data = {}
    
    # Load datasets into a dictionary
    for symbol in [
                'BTCUSDT',
                # 'ETHUSDT',
                #'SOLUSDT',
                #'ICPUSDT',
                #'AVAXUSDT'
            ]:
        for timeframe in timeframe_array:
            file_path = os.path.join(DATA_PATH, f'{symbol}_{timeframe}_data.csv')
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, parse_dates=['timestamp'])
                data[(symbol, timeframe)] = df.set_index('timestamp')
    
    for (symbol, timeframe), df in data.items():
        higher_timeframes = [tf for tf in timeframe_array 
                             if pd.Timedelta(timeframe_mapping[tf]) > pd.Timedelta(timeframe_mapping[timeframe])]
        
        for ht in higher_timeframes:
            if (symbol, ht) in data:
                ht_df = data[(symbol, ht)].reindex(df.index, method='ffill')
                df[f'{ht}_open'] = ht_df['open']
                df[f'{ht}_high'] = ht_df['high']
                df[f'{ht}_low'] = ht_df['low']
                # Use the current close from the lower timeframe (since we don't know the higher timeframe's final close)
                df[f'{ht}_close'] = df['close']
        
        output_path = os.path.join(DATA_PATH, f'{symbol}_{timeframe}_data.csv')
        df.reset_index().to_csv(output_path, index=False)
        print(f'Saved merged file: {output_path}')
def check_missing_intervals():
    for timeframe, freq in timeframe_mapping.items():
        for symbol in symbol_array:
            file_path = os.path.join(DATA_PATH, f'{symbol}_{timeframe}_data.csv')
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, parse_dates=['timestamp'])
                df = df.set_index('timestamp')
                
                # Check for duplicates
                duplicates = df.index.duplicated().sum()
                if duplicates > 0:
                    print(f'{symbol}_{timeframe}: {duplicates} duplicate timestamps found.')
                
                # Check for missing intervals
                all_times = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
                missing_times = all_times.difference(df.index)
                if not missing_times.empty:
                    print(f'{symbol}_{timeframe}: {len(missing_times)} missing timestamps.')
                else:
                    print(f'{symbol}_{timeframe}: No missing timestamps.')
def find_missing_timestamps(df_name):
    # Extract symbol and timeframe from dataframe name (e.g., 'BTCUSDT_1m')
    symbol_timeframe = df_name.replace('_data', '').strip()  # Clean suffix if any
    symbol, timeframe = symbol_timeframe.split('_')
    
    
    file_path = f'data/exports/{df_name}.csv'
    if not os.path.exists(file_path):
        print(f"File {file_path} not found.")
        return None

    df = pd.read_csv(file_path, parse_dates=['timestamp']).set_index('timestamp')

    if timeframe not in timeframe_mapping:
        print(f"Timeframe {timeframe} not recognized.")
        return None

    # Create a full range of timestamps for the timeframe
    freq = timeframe_mapping[timeframe]
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)

    # Find missing timestamps
    missing_timestamps = full_range.difference(df.index)

    # Create a dataframe for the missing timestamps
    missing_df = pd.DataFrame({'timestamp': missing_timestamps})
    
    if missing_df.empty:
        print(f"No missing timestamps for {df_name}.")
    else:
        print(f"Found {len(missing_df)} missing timestamps for {df_name}.")

    return missing_df

def insert_indicator_values(df):
    df['RSI'], df['RSI_MA'] = ti.calculate_rsi_with_ma(df['close'], rsi_period=14, ma_type="SMA", ma_length=14)
    df['MACD'], df['Signal'], df['MACD_Hist'] = ti.calculate_macd(df['close'], fast_period=12, slow_period=26, signal_period=9)
    df['BB_Mid'], df['BB_Upper'], df['BB_Lower'] = ti.calculate_bollinger_bands(df['close'], window=20, num_std=2)
    df['Stoch_K'], df['Stoch_D'] = ti.calculate_stochastic(df, k_period=14, d_period=3)
    df = ti.calculate_fibonacci_from_swings(df = df, suffix='_val')

In [9]:
def calculate_indicators(df, col_prefix):
    """Calculate indicators and add them to the dataframe with a prefix."""
    try:
        # Handle base timeframe (no prefix) and higher timeframes separately
        close_col = f'{col_prefix}_close' if col_prefix else 'close'

        if close_col not in df.columns:
            logging.warning(f"Column {close_col} not found in dataframe. Skipping indicator calculation.")
            return df

        df[f'{col_prefix}_RSI'], df[f'{col_prefix}_RSI_MA'] = ti.calculate_rsi_with_ma(df[close_col], rsi_period=14, ma_type="SMA", ma_length=14)
        df[f'{col_prefix}_MACD'], df[f'{col_prefix}_Signal'], df[f'{col_prefix}_MACD_Hist'] = ti.calculate_macd(df[close_col], fast_period=12, slow_period=26, signal_period=9)
        df[f'{col_prefix}_BB_Mid'], df[f'{col_prefix}_BB_Upper'], df[f'{col_prefix}_BB_Lower'] = ti.calculate_bollinger_bands(df[close_col], window=20, num_std=2)
        df[f'{col_prefix}_Stoch_K'], df[f'{col_prefix}_Stoch_D'] = ti.calculate_stochastic(df, k_period=14, d_period=3)
    
    except Exception as e:
        logging.error(f"Error calculating indicators for {col_prefix or 'base timeframe'}: {e}")

    return df


In [10]:
# Function to process each dataset
def process_symbol_timeframe(symbol, timeframe):
    """Processes a specific symbol and timeframe by computing all indicators."""
    
    if stop_flag.is_set():
        return
    file_path = os.path.join(DATA_PATH, f"{symbol}_{timeframe}_data.csv")
    if not os.path.exists(file_path):
        logging.warning(f"File {file_path} does not exist. Skipping.")
        return
    start_time = time.time()
    logging.info(f"Processing {symbol} on {timeframe} timeframe...")
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    df = calculate_indicators(df, "")
    for ht in ['15m', '1h', '4h', '1d', '1w']:
        if f"{ht}_close" in df.columns:
            df = calculate_indicators(df, ht)
    df.to_csv(file_path, index=False)
    logging.info(f"Completed {symbol} on {timeframe} in {time.time() - start_time:.2f}s")




In [11]:
def run_processing():
    """Runs multi-threaded processing for all symbols and timeframes."""
    global stop_flag
    stop_flag.clear()  # Ensure flag is not set before starting
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=4, thread_name_prefix="Worker") as executor:
            futures = []
            for symbol in symbol_array:
                for timeframe in timeframe_array:
                    if stop_flag.is_set():
                        break  # Stop if requested
                    futures.append(executor.submit(process_symbol_timeframe, symbol, timeframe))
            
            # Wait for all threads to complete unless stop_flag is set
            for future in concurrent.futures.as_completed(futures):
                if stop_flag.is_set():
                    break
                future.result()

# Extract

In [29]:
for symbol in symbol_array:
        for interval in timeframe_array:
            print(f"📊 Processing {symbol} {interval}")
            export_or_update(symbol, interval)
            time.sleep(2)

📊 Processing BTCUSDT 1m
✅ BTCUSDT 1m: Existing CSV found. Checking for updates...
📅 Last recorded timestamp: 2021-04-24 06:03:00
BTCUSDT 1m | Iteration 1: Latest timestamp: 2021-04-24 22:42:00, Total new records: 1000, Elapsed: 0.74s
BTCUSDT 1m | Iteration 2: Latest timestamp: 2021-04-25 20:06:00, Total new records: 2000, Elapsed: 1.98s
BTCUSDT 1m | Iteration 3: Latest timestamp: 2021-04-26 12:46:00, Total new records: 3000, Elapsed: 3.21s
BTCUSDT 1m | Iteration 4: Latest timestamp: 2021-04-27 05:26:00, Total new records: 4000, Elapsed: 4.51s
BTCUSDT 1m | Iteration 5: Latest timestamp: 2021-04-27 22:06:00, Total new records: 5000, Elapsed: 5.77s
BTCUSDT 1m | Iteration 6: Latest timestamp: 2021-04-28 14:46:00, Total new records: 6000, Elapsed: 7.06s
BTCUSDT 1m | Iteration 7: Latest timestamp: 2021-04-29 07:26:00, Total new records: 7000, Elapsed: 8.38s
BTCUSDT 1m | Iteration 8: Latest timestamp: 2021-04-30 00:06:00, Total new records: 8000, Elapsed: 9.68s
BTCUSDT 1m | Iteration 9: Lates

# Insert higher timeframes

In [ ]:
# merge_timeframes()
check_missing_intervals()

In [ ]:
missing_df = find_missing_timestamps('BTCUSDT_1d_data')
if missing_df is not None and not missing_df.empty:
    missing_df.to_csv('BTCUSDT_1d_missing_timestamps.csv', index=False)

# Development

In [ ]:
df = pd.read_csv('data/exports/BTCUSDT_1d_data.csv', parse_dates=['timestamp'])

In [ ]:
df.head()

In [ ]:
df_with_fib = ti.calculate_fibonacci_from_swings(df = df, suffix='_1d')

In [ ]:
df_with_fib.tail(5)

In [ ]:
pl.plot_fibonacci_chart(df_with_fib, selected_index=df_with_fib.index[-1], title="Fibonacci Retracement - 1D", suffix="_d")

In [26]:
df = pd.read_csv('data/exports/BTCUSDT_1m_data.csv', parse_dates=['timestamp'])

In [27]:
df['timestamp'].min()

Timestamp('2019-01-01 00:00:00')

# Archive

In [ ]:
"""
EXPORT HISTORY

symbol = "BTCUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)

symbol = "ETHUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)

symbol = "SOLUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)

symbol = "ICPUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)

symbol = "AVAXUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)
"""

# One time code

In [12]:
# load data into database
import pandas as pd
import business.utils.db_util as db_util  # make sure this file is in your working directory or in the PYTHONPATH
import glob

In [13]:
def get_columns_definitions(df):
    """
    Map DataFrame dtypes to SQL column definitions.
    
    Supported mappings:
      - int64 -> BIGINT
      - float64 -> NUMERIC
      - object -> TEXT
      - bool -> BOOLEAN
      - datetime64[ns] -> TIMESTAMP
    """
    dtype_mapping = {
        'int64': 'BIGINT',
        'float64': 'NUMERIC',
        'object': 'TEXT',
        'bool': 'BOOLEAN',
        'datetime64[ns]': 'TIMESTAMP'
    }
    
    columns_defs = []
    for col in df.columns:
        col_dtype = str(df[col].dtype)
        sql_type = dtype_mapping.get(col_dtype, 'TEXT')
        columns_defs.append(f'"{col}" {sql_type}')
    return columns_defs

In [16]:
def process_file(file_path):
    try:
        print(f"Processing file: {file_path}")
        # Read CSV file into DataFrame
        df = pd.read_csv(file_path)
        print(f"Loaded DataFrame with {df.shape[0]} rows and {df.shape[1]} columns.")

        # Convert columns to proper datetime formats if they exist
        if "timestamp" in df.columns:
            df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
        if "close_time" in df.columns:
            # Convert Unix epoch in milliseconds to datetime
            df["close_time"] = pd.to_datetime(df["close_time"], unit='ms', errors="coerce")

        # Create table name from file name (without extension) and remove '_data' if exists
        file_name = os.path.basename(file_path)
        table_name = os.path.splitext(file_name)[0].replace('_data', '').lower()
        print(f"Creating table: {table_name}")

        # Generate column definitions and create table
        columns_definitions = get_columns_definitions(df)
        db_util.create_table(table_name, columns_definitions)
        print(f"Table {table_name} created successfully.")

        # Add a unique constraint on the "timestamp" column
        db_util.add_unique_constraint(table_name, "timestamp")

        # Insert DataFrame data into the created table in batches
        print(f"Inserting data into table: {table_name}")
        db_util.load_dataframe_to_database(df, table_name, batch_size=10000)
        print(f"Data loaded successfully into table: {table_name}\n")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

In [14]:
csv_files = glob.glob(os.path.join(DATA_PATH, '*.csv'))

if not csv_files:
    print("No CSV files found in the folder.")
else:
    print(f"Found {len(csv_files)} CSV files.")
    for file_path in csv_files:
        try:
            print(f"Processing file: {file_path}")
            # Read CSV file into DataFrame
            df = pd.read_csv(file_path)
            print(f"Loaded DataFrame with {df.shape[0]} rows and {df.shape[1]} columns.")

            # Convert columns to proper datetime formats if they exist
            if "timestamp" in df.columns:
                df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
            if "close_time" in df.columns:
                # Convert Unix epoch in milliseconds to datetime
                df["close_time"] = pd.to_datetime(df["close_time"], unit='ms', errors="coerce")

            # Create table name from file name (without extension) and remove '_data' if exists
            file_name = os.path.basename(file_path)
            table_name = os.path.splitext(file_name)[0].replace('_data', '').lower()
            print(f"Creating table: {table_name}")

            # Generate column definitions (e.g., "col1 INTEGER, col2 TEXT")
            columns_definitions = get_columns_definitions(df)
            db_util.create_table(table_name, columns_definitions)
            print(f"Table {table_name} created successfully.")
            
            db_util.add_unique_constraint(table_name, "timestamp")

            # Load DataFrame data into the created table
            print(f"Inserting data into table: {table_name}")
            db_util.load_dataframe_to_database(df, table_name, batch_size = 10000)
            print(f"Data loaded successfully into table: {table_name}\n")

        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

Found 30 CSV files.
Processing file: data/exports\AVAXUSDT_15m_data.csv
Loaded DataFrame with 155652 rows and 78 columns.
Creating table: avaxusdt_15m
Table avaxusdt_15m created successfully.
Unique constraint 'avaxusdt_15m_timestamp_unique' added on column 'timestamp' in table 'avaxusdt_15m'.
Inserting data into table: avaxusdt_15m


KeyboardInterrupt: 

In [17]:
csv_files = glob.glob(os.path.join(DATA_PATH, '*.csv'))

if not csv_files:
    print("No CSV files found in the folder.")
else:
    print(f"Found {len(csv_files)} CSV files.")
    # Using a thread pool to process files concurrently.
    with concurrent.futures.ThreadPoolExecutor(max_workers=4, thread_name_prefix="Worker") as executor:
        futures = [executor.submit(process_file, file_path) for file_path in csv_files]
        # Optionally, wait for all threads to complete.
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error in thread: {e}")

Found 30 CSV files.
Processing file: data/exports\AVAXUSDT_15m_data.csv
Processing file: data/exports\AVAXUSDT_1d_data.csv
Processing file: data/exports\AVAXUSDT_1h_data.csv
Processing file: data/exports\AVAXUSDT_1m_data.csv
Loaded DataFrame with 1623 rows and 36 columns.
Creating table: avaxusdt_1d
Table avaxusdt_1d created successfully.
Unique constraint 'avaxusdt_1d_timestamp_unique' added on column 'timestamp' in table 'avaxusdt_1d'.
Inserting data into table: avaxusdt_1d
Data loaded successfully into table: avaxusdt_1d

Processing file: data/exports\AVAXUSDT_1w_data.csv
Loaded DataFrame with 232 rows and 22 columns.
Creating table: avaxusdt_1w
Table avaxusdt_1w created successfully.
Unique constraint 'avaxusdt_1w_timestamp_unique' added on column 'timestamp' in table 'avaxusdt_1w'.
Inserting data into table: avaxusdt_1w
Data loaded successfully into table: avaxusdt_1w

Processing file: data/exports\AVAXUSDT_4h_data.csv
Loaded DataFrame with 9735 rows and 50 columns.
Creating table

In [15]:
df

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,1h_open,1h_high,1h_low,1h_close,4h_open,4h_high,4h_low,4h_close,1d_open,1d_high,1d_low,1d_close,1w_open,1w_high,1w_low,1w_close,_RSI,_RSI_MA,_MACD,_Signal,_MACD_Hist,_BB_Mid,_BB_Upper,_BB_Lower,_Stoch_K,_Stoch_D,1h_RSI,1h_RSI_MA,1h_MACD,1h_Signal,1h_MACD_Hist,1h_BB_Mid,1h_BB_Upper,1h_BB_Lower,1h_Stoch_K,1h_Stoch_D,4h_RSI,4h_RSI_MA,4h_MACD,4h_Signal,4h_MACD_Hist,4h_BB_Mid,4h_BB_Upper,4h_BB_Lower,4h_Stoch_K,4h_Stoch_D,1d_RSI,1d_RSI_MA,1d_MACD,1d_Signal,1d_MACD_Hist,1d_BB_Mid,1d_BB_Upper,1d_BB_Lower,1d_Stoch_K,1d_Stoch_D,1w_RSI,1w_RSI_MA,1w_MACD,1w_Signal,1w_MACD_Hist,1w_BB_Mid,1w_BB_Upper,1w_BB_Lower,1w_Stoch_K,1w_Stoch_D
0,2020-09-22 06:30:00,0.8500,6.0000,0.8500,5.4000,2047137.84,1600757099999,9.302422e+06,16557,1076876.97,4.942199e+06,0,0.85,6.00,0.850,5.4000,0.85,6.0,0.85,5.4000,0.85,7.0,0.85,5.4000,0.85,7.0,0.85,5.4000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN
1,2020-09-22 06:45:00,5.3997,5.6508,4.6000,4.8811,1151234.83,1600757999999,5.975189e+06,13147,552359.56,2.884919e+06,0,0.85,6.00,0.850,4.8811,0.85,6.0,0.85,4.8811,0.85,7.0,0.85,4.8811,0.85,7.0,0.85,4.8811,NaN,NaN,-0.041394,-0.008279,-0.033115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.041394,-0.008279,-3.311499e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.041394,-0.008279,-3.311499e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.041394,-0.008279,-3.311499e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.041394,-0.008279,-3.311499e-02,NaN,NaN,NaN,NaN,NaN
2,2020-09-22 07:00:00,4.8800,4.9336,4.2450,4.4593,973865.08,1600758899999,4.401221e+06,10904,465161.68,2.102820e+06,0,4.88,5.35,4.245,4.4593,0.85,6.0,0.85,4.4593,0.85,7.0,0.85,4.4593,0.85,7.0,0.85,4.4593,NaN,NaN,-0.107001,-0.028023,-0.078978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.107001,-0.028023,-7.897769e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.107001,-0.028023,-7.897769e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.107001,-0.028023,-7.897769e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.107001,-0.028023,-7.897769e-02,NaN,NaN,NaN,NaN,NaN
3,2020-09-22 07:15:00,4.4497,5.3500,4.4401,4.9312,940771.21,1600759799999,4.705748e+06,10555,495326.46,2.484269e+06,0,4.88,5.35,4.245,4.9312,0.85,6.0,0.85,4.9312,0.85,7.0,0.85,4.9312,0.85,7.0,0.85,4.9312,NaN,NaN,-0.119539,-0.046326,-0.073212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.119539,-0.046326,-7.321237e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.119539,-0.046326,-7.321237e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.119539,-0.046326,-7.321237e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.119539,-0.046326,-7.321237e-02,NaN,NaN,NaN,NaN,NaN
4,2020-09-22 07:30:00,4.9141,5.1500,4.5662,4.6118,674186.91,1600760699999,3.316496e+06,8024,400434.67,1.975965e+06,0,4.88,5.35,4.245,4.6118,0.85,6.0,0.85,4.6118,0.85,7.0,0.85,4.6118,0.85,7.0,0.85,4.6118,NaN,NaN,-0.153479,-0.067757,-0.085722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.153479,-0.067757,-8.572187e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.153479,-0.067757,-8.572187e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.153479,-0.067757,-8.572187e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.153479,-0.067757,-8.572187e-02,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155647,2025-03-02 13:45:00,21.9100,22.0200,21.7900,22.0200,73103.39,1740923999999,1.601754e+06,2773,52581.12,1.152345e+06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.983912,52.579320,0.010998,0.020492,-0.009494,21.9995,22.130965,21.868035,76.666667,44.910394,NaN,NaN,-0.105738,-0.105738,5.551115e-17,NaN,NaN,NaN,76.666667,44.910394,NaN,NaN,-0.10